In [75]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [177]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [178]:
dataa.columns

Index(['instance_id', 'item_category_list_num', 'item_property_list_num',
       'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click'],
      dtype='object')

In [179]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum', 'shop_click']

In [180]:
dataa = dataa[tezheng]

In [181]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [182]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 35)
2018-09-24 23:59:47
(357066, 35)
(121021, 35)


In [183]:

print('test')
test_a = pd.read_csv('test.csv',index_col=False)
data2a = pd.read_csv('test_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [184]:
data2a = data2a[tezheng]

In [185]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [186]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']

(18371, 30)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [187]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [188]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(18371, 33)


In [189]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_id_pvnum', 'occupation_click',
       'category_1_pvnum', 'shop_click', 'category_0', 'category_1',
       'category_2', 'property_0', 'property_1', 'property_2', 'halfhour',
       'user_count', 'item_count'],
      dtype='object')
(496458, 586)


In [190]:
# LogisticRegression?

In [191]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [192]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [193]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
#                         max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683896
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.67484
[3]	valid_0's binary_logloss: 0.665952
[4]	valid_0's binary_logloss: 0.657236
[5]	valid_0's binary_logloss: 0.648689
[6]	valid_0's binary_logloss: 0.640305
[7]	valid_0's binary_logloss: 0.632077
[8]	valid_0's binary_logloss: 0.624017
[9]	valid_0's binary_logloss: 0.616093
[10]	valid_0's binary_logloss: 0.608318
[11]	valid_0's binary_logloss: 0.600687
[12]	valid_0's binary_logloss: 0.593199
[13]	valid_0's binary_logloss: 0.585843
[14]	valid_0's binary_logloss: 0.578623
[15]	valid_0's binary_logloss: 0.571523
[16]	valid_0's binary_logloss: 0.564546
[17]	valid_0's binary_logloss: 0.557693
[18]	valid_0's binary_logloss: 0.550961
[19]	valid_0's binary_logloss: 0.544354
[20]	valid_0's binary_logloss: 0.537857
[21]	valid_0's binary_logloss: 0.531473
[22]	valid_0's binary_logloss: 0.525188
[23]	valid_0's binary_logloss: 0.519015
[24]	valid_0's binary_loglo

[208]	valid_0's binary_logloss: 0.122975
[209]	valid_0's binary_logloss: 0.122457
[210]	valid_0's binary_logloss: 0.121943
[211]	valid_0's binary_logloss: 0.121438
[212]	valid_0's binary_logloss: 0.120941
[213]	valid_0's binary_logloss: 0.120448
[214]	valid_0's binary_logloss: 0.119961
[215]	valid_0's binary_logloss: 0.119478
[216]	valid_0's binary_logloss: 0.119006
[217]	valid_0's binary_logloss: 0.118538
[218]	valid_0's binary_logloss: 0.118075
[219]	valid_0's binary_logloss: 0.117617
[220]	valid_0's binary_logloss: 0.117168
[221]	valid_0's binary_logloss: 0.116724
[222]	valid_0's binary_logloss: 0.116285
[223]	valid_0's binary_logloss: 0.115849
[224]	valid_0's binary_logloss: 0.115423
[225]	valid_0's binary_logloss: 0.115001
[226]	valid_0's binary_logloss: 0.114582
[227]	valid_0's binary_logloss: 0.114169
[228]	valid_0's binary_logloss: 0.113762
[229]	valid_0's binary_logloss: 0.113359
[230]	valid_0's binary_logloss: 0.11296
[231]	valid_0's binary_logloss: 0.112566
[232]	valid_0's b

[410]	valid_0's binary_logloss: 0.085286
[411]	valid_0's binary_logloss: 0.0852502
[412]	valid_0's binary_logloss: 0.085212
[413]	valid_0's binary_logloss: 0.0851752
[414]	valid_0's binary_logloss: 0.0851403
[415]	valid_0's binary_logloss: 0.0851078
[416]	valid_0's binary_logloss: 0.0850715
[417]	valid_0's binary_logloss: 0.0850358
[418]	valid_0's binary_logloss: 0.0850016
[419]	valid_0's binary_logloss: 0.0849696
[420]	valid_0's binary_logloss: 0.0849371
[421]	valid_0's binary_logloss: 0.0849045
[422]	valid_0's binary_logloss: 0.0848749
[423]	valid_0's binary_logloss: 0.0848416
[424]	valid_0's binary_logloss: 0.0848091
[425]	valid_0's binary_logloss: 0.0847806
[426]	valid_0's binary_logloss: 0.0847501
[427]	valid_0's binary_logloss: 0.0847199
[428]	valid_0's binary_logloss: 0.0846926
[429]	valid_0's binary_logloss: 0.0846649
[430]	valid_0's binary_logloss: 0.084635
[431]	valid_0's binary_logloss: 0.0846053
[432]	valid_0's binary_logloss: 0.084577
[433]	valid_0's binary_logloss: 0.0845

[606]	valid_0's binary_logloss: 0.0825223
[607]	valid_0's binary_logloss: 0.0825188
[608]	valid_0's binary_logloss: 0.0825136
[609]	valid_0's binary_logloss: 0.0825099
[610]	valid_0's binary_logloss: 0.0825059
[611]	valid_0's binary_logloss: 0.0825028
[612]	valid_0's binary_logloss: 0.0824991
[613]	valid_0's binary_logloss: 0.0824929
[614]	valid_0's binary_logloss: 0.0824893
[615]	valid_0's binary_logloss: 0.0824865
[616]	valid_0's binary_logloss: 0.0824821
[617]	valid_0's binary_logloss: 0.0824769
[618]	valid_0's binary_logloss: 0.0824726
[619]	valid_0's binary_logloss: 0.0824685
[620]	valid_0's binary_logloss: 0.0824643
[621]	valid_0's binary_logloss: 0.0824558
[622]	valid_0's binary_logloss: 0.0824546
[623]	valid_0's binary_logloss: 0.0824521
[624]	valid_0's binary_logloss: 0.0824492
[625]	valid_0's binary_logloss: 0.0824438
[626]	valid_0's binary_logloss: 0.0824381
[627]	valid_0's binary_logloss: 0.0824345
[628]	valid_0's binary_logloss: 0.0824312
[629]	valid_0's binary_logloss: 0.

[805]	valid_0's binary_logloss: 0.0821119
[806]	valid_0's binary_logloss: 0.0821099
[807]	valid_0's binary_logloss: 0.0821074
[808]	valid_0's binary_logloss: 0.0821055
[809]	valid_0's binary_logloss: 0.0821055
[810]	valid_0's binary_logloss: 0.0821058
[811]	valid_0's binary_logloss: 0.0821065
[812]	valid_0's binary_logloss: 0.0821041
[813]	valid_0's binary_logloss: 0.082102
[814]	valid_0's binary_logloss: 0.0821011
[815]	valid_0's binary_logloss: 0.0821002
[816]	valid_0's binary_logloss: 0.0820982
[817]	valid_0's binary_logloss: 0.0820957
[818]	valid_0's binary_logloss: 0.0820943
[819]	valid_0's binary_logloss: 0.0820947
[820]	valid_0's binary_logloss: 0.0820938
[821]	valid_0's binary_logloss: 0.0820952
[822]	valid_0's binary_logloss: 0.0820955
[823]	valid_0's binary_logloss: 0.0820939
[824]	valid_0's binary_logloss: 0.0820961
[825]	valid_0's binary_logloss: 0.0820935
[826]	valid_0's binary_logloss: 0.0820924
[827]	valid_0's binary_logloss: 0.0820906
[828]	valid_0's binary_logloss: 0.0

[1004]	valid_0's binary_logloss: 0.0820188
[1005]	valid_0's binary_logloss: 0.0820181
[1006]	valid_0's binary_logloss: 0.0820178
[1007]	valid_0's binary_logloss: 0.0820182
[1008]	valid_0's binary_logloss: 0.0820196
[1009]	valid_0's binary_logloss: 0.082018
[1010]	valid_0's binary_logloss: 0.0820173
[1011]	valid_0's binary_logloss: 0.0820149
[1012]	valid_0's binary_logloss: 0.082013
[1013]	valid_0's binary_logloss: 0.0820121
[1014]	valid_0's binary_logloss: 0.0820132
[1015]	valid_0's binary_logloss: 0.0820131
[1016]	valid_0's binary_logloss: 0.0820104
[1017]	valid_0's binary_logloss: 0.0820093
[1018]	valid_0's binary_logloss: 0.0820098
[1019]	valid_0's binary_logloss: 0.0820091
[1020]	valid_0's binary_logloss: 0.0820088
[1021]	valid_0's binary_logloss: 0.0820081
[1022]	valid_0's binary_logloss: 0.0820089
[1023]	valid_0's binary_logloss: 0.0820092
[1024]	valid_0's binary_logloss: 0.0820095
[1025]	valid_0's binary_logloss: 0.0820092
[1026]	valid_0's binary_logloss: 0.0820085
[1027]	valid_

[1196]	valid_0's binary_logloss: 0.0819749
[1197]	valid_0's binary_logloss: 0.0819756
[1198]	valid_0's binary_logloss: 0.0819752
[1199]	valid_0's binary_logloss: 0.0819748
[1200]	valid_0's binary_logloss: 0.0819752
[1201]	valid_0's binary_logloss: 0.0819748
[1202]	valid_0's binary_logloss: 0.0819754
[1203]	valid_0's binary_logloss: 0.0819757
[1204]	valid_0's binary_logloss: 0.0819764
[1205]	valid_0's binary_logloss: 0.0819764
[1206]	valid_0's binary_logloss: 0.081976
[1207]	valid_0's binary_logloss: 0.0819765
[1208]	valid_0's binary_logloss: 0.081976
[1209]	valid_0's binary_logloss: 0.0819764
[1210]	valid_0's binary_logloss: 0.0819761
[1211]	valid_0's binary_logloss: 0.0819754
[1212]	valid_0's binary_logloss: 0.0819764
[1213]	valid_0's binary_logloss: 0.0819749
[1214]	valid_0's binary_logloss: 0.0819756
[1215]	valid_0's binary_logloss: 0.0819775
[1216]	valid_0's binary_logloss: 0.081977
[1217]	valid_0's binary_logloss: 0.0819783
[1218]	valid_0's binary_logloss: 0.0819786
[1219]	valid_0

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [194]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0803019923777
0.0819743468339


In [196]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [197]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180415.txt',sep=" ",index=False)